# SnapTo3D Spatial‑Planning Pipeline

Flow: 
```
USER prompt -> LLM (scene‑graph JSON) -> Constraint Solver -> Blender CodeGen -> Render -> VLM Verifier
```

## 0  Setup and dependencies

In [124]:
# ↳ Run once per environment
!source spatial-env/bin/activate
# !pip install -r requirements.txt

## 1  Scene‑graph schema & helper classes

In [125]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv
# from helper import save_scene_graph


load_dotenv()

True

## 2  LLM reasoning → JSON

In [126]:
MODEL = 'gpt-4'

OpenAPIClient = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_to_scene(prompt: str) -> dict:
    """Call GPT with function‑calling and return a scene‑graph dict."""
    function = {
    "name": "create_scene_graph",
    "description": "Creates a comprehensive 3D scene graph with room architecture, furniture, and materials from a description",
    "parameters": {
        "type": "object",
        "properties": {
            "metadata": {
                "type": "object",
                "description": "Scene-level metadata",
                "properties": {
                    "projectId": {"type": "string", "description": "Unique scene/room ID"},
                    "roomType": {"type": "string", "enum": ["bedroom", "lounge", "bathroom", "kitchen", "office", "dining"], "description": "Type of room"},
                    "units": {"type": "string", "description": "Linear unit symbol (e.g., 'm', 'ft')"},
                    "axes": {
                        "type": "object",
                        "properties": {
                            "right": {"type": "string", "description": "Axis pointing right (e.g., 'X+')"},
                            "forward": {"type": "string", "description": "Axis pointing forward (e.g., 'Y+')"},
                            "up": {"type": "string", "description": "Axis pointing up (e.g., 'Z+')"}
                        },
                        "required": ["right", "forward", "up"]
                    }
                },
                "required": ["projectId", "roomType", "units", "axes"]
            },
            "room": {
                "type": "object",
                "description": "Architectural shell definition",
                "properties": {
                    "dims": {
                        "type": "object",
                        "properties": {
                            "w": {"type": "number", "description": "Interior width"},
                            "d": {"type": "number", "description": "Interior depth"},
                            "h": {"type": "number", "description": "Interior height"}
                        },
                        "required": ["w", "d", "h"]
                    },
                    "walls": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "id": {"type": "string", "description": "Stable wall identifier"},
                                "start": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "World-space XYZ of wall start"},
                                "end": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "World-space XYZ of wall end"},
                                "thickness": {"type": "number", "description": "Wall thickness"},
                                "mat": {"type": "string", "description": "Material ID"},
                                "mesh": {"type": ["string", "null"], "description": "Optional custom mesh path"}
                            },
                            "required": ["id", "start", "end", "thickness", "mat"]
                        }
                    },
                    "openings": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "id": {"type": "string", "description": "Door/window identifier"},
                                "wallId": {"type": "string", "description": "Host wall's ID"},
                                "offset": {"type": "number", "description": "Distance from wall start"},
                                "w": {"type": "number", "description": "Opening width"},
                                "h": {"type": "number", "description": "Opening height"},
                                "sill": {"type": "number", "description": "Z-offset of bottom edge"}
                            },
                            "required": ["id", "wallId", "offset", "w", "h", "sill"]
                        }
                    },
                    "floorMat": {"type": "string", "description": "Default floor material ID"}
                },
                "required": ["dims", "walls", "floorMat"]
            },
            "furniture": {
                "type": "array",
                "description": "Furnishings and decor objects",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {"type": "string", "description": "Instance name"},
                        "catalogId": {"type": "string", "description": "SKU/external reference"},
                        "category": {"type": "string", "description": "Furniture category (e.g., bed, sofa, sink)"},
                        "dims": {
                            "type": ["object", "null"],
                            "properties": {
                                "w": {"type": "number"},
                                "d": {"type": "number"},
                                "h": {"type": "number"},
                                "source": {"type": "string", "enum": ["catalog", "inferred", "user"]}
                            }
                        },
                        "xform": {
                            "type": "object",
                            "description": "Blender-ready transform",
                            "properties": {
                                "pos": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "Translation XYZ"},
                                "rot": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "Euler rotation degrees XYZ"},
                                "scale": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "Non-uniform scale factors"}
                            },
                            "required": ["pos", "rot", "scale"]
                        },
                        "mesh": {"type": ["string", "null"], "description": "File path to mesh/prefab"},
                        "matOverride": {"type": ["string", "null"], "description": "Optional material swap"},
                        "tags": {"type": "array", "items": {"type": "string"}, "description": "Semantic labels"}
                    },
                    "required": ["id", "catalogId", "category", "xform"]
                }
            },
            "constraints": {
                "type": "array",
                "description": "Declarative layout rules",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {"type": "string", "description": "Human-readable slug"},
                        "type": {"type": "string", "enum": ["offset", "align", "inside", "groupAlign"], "description": "Constraint type"},
                        "targets": {"type": "array", "items": {"type": "string"}, "description": "IDs of objects/walls involved"},
                        "axis": {"type": "string", "enum": ["X", "Y", "Z"], "description": "Axis for offset/align rules"},
                        "distance": {"type": "number", "description": "Gap size"},
                        "relativeTo": {"type": "string", "description": "Anchoring object ID"},
                        "alignAxis": {"type": "string", "description": "Axis to match in groupAlign"}
                    },
                    "required": ["id", "type", "targets"]
                }
            },
            "materials": {
                "type": "array",
                "description": "Shared material palette",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {"type": "string", "description": "Material name"},
                        "color": {"type": "array", "items": {"type": "number", "minimum": 0, "maximum": 1}, "minItems": 3, "maxItems": 3, "description": "RGB 0-1 values"},
                        "tex": {"type": ["string", "null"], "description": "Path to texture map"},
                        "roughness": {"type": ["number", "null"], "minimum": 0, "maximum": 1, "description": "PBR roughness 0-1"},
                        "scaleUV": {"type": ["number", "null"], "description": "Texture tiling factor"}
                    },
                    "required": ["id"]
                }
            }
        },
        "required": ["metadata", "room", "furniture", "materials"]
    }
}
    
    system_message = """
                        You are a 3D spatial planning assistant. 
                        Generate realistic room layouts with proper furniture placement, considering spatial relationships, accessibility, and design principles.
                        Always include complete scene graphs with dimensions, transforms, and materials.
        """
    
    try:
        response = OpenAPIClient.chat.completions.create(
            model=MODEL,
            messages=[
                {'role':'system', 'content': system_message},
                {'role':'user', 'content': prompt}
            ],
            functions=[function],
            function_call={'name':'create_scene_graph'}
        )
        
        if not response.choices or not response.choices[0].message.function_call:
            raise ValueError("No valid response from API")
            
        print('response', response.choices[0].message)
        return json.loads(response.choices[0].message.function_call.arguments)
    except Exception as e:
        print(f"Error in llm_to_scene: {str(e)}")
        raise

## 3  Constraint solver → absolute coordinates

In [127]:
from ortools.sat.python import cp_model
import time

GRID = 100 
MAX_RETRIES = 5

def solve_scene(scene: dict) -> dict:
    """
    Solve scene constraints. Returns placements dict on success, False on error.
    """
    try:
        m = cp_model.CpModel()
        pos, half = {}, {}

        print('scene', scene)

        if scene.get('constraints') is None:
            print("Error: No constraints found in scene")
            return False

        # Extract furniture items and convert to expected format
        # centres (x,y) in 1 cm grid
        for item in scene['furniture']:
            # Handle different dimension formats
            if 'w' in item['dims'] and 'l' in item['dims']:
                w = item['dims']['w']
                d = item['dims']['l']  # 'l' (length) maps to 'd' (depth)
            elif 'balloon' in item['dims']:  # Special case for lamp
                w = item['dims']['balloon']['r'] * 2
                d = item['dims']['balloon']['r'] * 2
            else:
                w, d = 0.5, 0.5  # Default size
                
            half[item['id']] = (int(w*GRID/2), int(d*GRID/2))
            pos[item['id']]  = (                       # centre coordinates
                m.NewIntVar(0, int(scene['room']['dims']['w']*GRID), f'x_{item["id"]}'),
                m.NewIntVar(0, int(scene['room']['dims']['d']*GRID), f'y_{item["id"]}')
            )

        # Get wall positions for offset constraints
        wall_positions = {}
        for wall in scene['room']['walls']:
            if wall['id'] == 'wall1':  # Bottom wall (Y=0)
                wall_positions[wall['id']] = {'y': 0}
            elif wall['id'] == 'wall2':  # Left wall (X=0)
                wall_positions[wall['id']] = {'x': 0}
            elif wall['id'] == 'wall3':  # Right wall
                wall_positions[wall['id']] = {'x': scene['room']['dims']['w']}
            elif wall['id'] == 'wall4':  # Top wall
                wall_positions[wall['id']] = {'y': scene['room']['dims']['d']}

        # relational constraints --------------------------------------------------
        for c in scene['constraints']:
            if c['type'] == 'offset':
                # Handle offset from wall constraints
                for target in c['targets']:
                    if target not in pos:
                        print(f"Error: Target '{target}' not found in furniture")
                        return False
                        
                    distance = int(c['distance'] * GRID)
                    x, y = pos[target]
                    w, d = half[target]
                    
                    if c['relativeTo'] not in wall_positions:
                        print(f"Error: Wall '{c['relativeTo']}' not found")
                        return False
                        
                    wall_pos = wall_positions[c['relativeTo']]
                    
                    if c['axis'] == 'X' and 'x' in wall_pos:
                        if wall_pos['x'] == 0:  # Left wall
                            m.Add(x - w == distance)
                        else:  # Right wall
                            m.Add(x + w == int(wall_pos['x'] * GRID) - distance)
                    elif c['axis'] == 'Y' and 'y' in wall_pos:
                        if wall_pos['y'] == 0:  # Bottom wall
                            m.Add(y - d == distance)
                        else:  # Top wall
                            m.Add(y + d == int(wall_pos['y'] * GRID) - distance)
                            
            elif c['type'] == 'align':
                # Handle alignment constraints
                for target in c['targets']:
                    if target not in pos:
                        print(f"Error: Target '{target}' not found in furniture")
                        return False
                    if c['relativeTo'] not in pos:
                        print(f"Error: RelativeTo '{c['relativeTo']}' not found in furniture")
                        return False
                        
                    xa, ya = pos[target]
                    xb, yb = pos[c['relativeTo']]
                    
                    if c.get('alignAxis') == 'Y':
                        # Align along Y axis (same Y coordinate)
                        m.Add(ya == yb)
                    elif c.get('alignAxis') == 'X':
                        # Align along X axis (same X coordinate)
                        m.Add(xa == xb)

        # NON-OVERLAP (boolean disjunction) ---------------------------------------
        ids = list(pos.keys())
        for i in range(len(ids)):
            for j in range(i+1, len(ids)):
                A, B = ids[i], ids[j]
                xa, ya = pos[A]; xb, yb = pos[B]
                wa, da = half[A]; wb, db = half[B]

                # Skip overlap check for lamp on desk (special case)
                if (A == 'lamp1' and B == 'desk1') or (A == 'desk1' and B == 'lamp1'):
                    continue

                left   = m.NewBoolVar(f'{A}_left_{B}')
                right  = m.NewBoolVar(f'{A}_right_{B}')
                front  = m.NewBoolVar(f'{A}_front_{B}')
                behind = m.NewBoolVar(f'{A}_behind_{B}')

                m.Add(xa + wa <= xb - wb).OnlyEnforceIf(left)
                m.Add(xb + wb <= xa - wa).OnlyEnforceIf(right)
                m.Add(ya + da <= yb - db).OnlyEnforceIf(front)
                m.Add(yb + db <= ya - da).OnlyEnforceIf(behind)

                # at least one spatial separation must hold
                m.AddBoolOr([left, right, front, behind])

        # -------------------------------------------------------------------------
        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = 10.0  # Add timeout
        status = solver.Solve(m)
        
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            return {oid: (solver.Value(x)/GRID, solver.Value(y)/GRID)
                    for oid,(x,y) in pos.items()}
        else:
            print(f"Error: Solver failed with status: {solver.StatusName(status)}")
            if status == cp_model.INFEASIBLE:
                print("The constraints are contradictory and cannot be satisfied.")
            return False
            
    except Exception as e:
        print(f"Error in solve_scene: {str(e)}")
        return False

## 4  CodeGen → Blender .py script

In [128]:
def gen_blender_py(scene: dict, placements: dict, outfile='assets/blender/script.py'):
    """
    Generate Blender Python script from scene and placements.
    
    Args:
        scene: Scene dictionary with room and furniture info
        placements: Dictionary mapping furniture IDs to (x, y) positions
        outfile: Output filename for the Python script
    """
    lines = [
        "import bpy",
        "from mathutils import Vector",
        "",
        "# Clear existing objects",
        "bpy.ops.object.select_all(action='SELECT')",
        "bpy.ops.object.delete()",
        "",
    ]

    # Get room dimensions
    room_dims = scene.get('room', {}).get('dims', {'w': 4, 'd': 2, 'h': 3})
    room_width = room_dims['w']
    room_depth = room_dims['d']
    room_height = room_dims['h']
    
    # Calculate camera position - behind and above the room
    cam_location = (
        room_width / 2,  # Center X
        -room_depth * 1.5,  # Behind the room (negative Z in your coordinate system)
        room_height * 1.2  # Above the room
    )
    cam_target = (room_width / 2, room_depth / 2, room_height / 2)

    # Add camera setup code
    lines += [
        f"# Add camera",
        f"cam_data = bpy.data.cameras.new(name='Camera')",
        f"cam_data.lens = 35",
        f"cam_object = bpy.data.objects.new('Camera', cam_data)",
        f"bpy.context.collection.objects.link(cam_object)",
        f"cam_object.location = {cam_location}",
        f"",
        f"# Point camera at room center",
        f"direction = Vector({cam_target}) - Vector({cam_location})",
        f"rot_quat = direction.to_track_quat('-Z', 'Y')",
        f"cam_object.rotation_euler = rot_quat.to_euler()",
        f"bpy.context.scene.camera = cam_object",
        "",
        "# Lighting",
        "bpy.ops.object.light_add(type='SUN', location=(5, -5, 10))",
        "bpy.context.active_object.data.energy = 2.0",
        "",
        "# Render settings",
        "bpy.context.scene.render.engine = 'CYCLES'",
        "bpy.context.scene.cycles.samples = 64",
        "bpy.context.scene.render.resolution_x = 1920",
        "bpy.context.scene.render.resolution_y = 1080",
        "",
    ]

    # Create room (floor and walls)
    lines += [
        "# Create floor",
        f"bpy.ops.mesh.primitive_cube_add(location=({room_width/2}, {room_depth/2}, -0.05))",
        f"floor = bpy.context.active_object",
        f"floor.name = 'Floor'",
        f"floor.scale = ({room_width/2}, {room_depth/2}, 0.05)",
        "",
    ]

    # Add walls based on wall definitions
    if 'walls' in scene.get('room', {}):
        lines.append("# Create walls")
        for i, wall in enumerate(scene['room']['walls']):
            wall_id = wall.get('id', f'wall{i+1}')
            start = wall['start']
            end = wall['end']
            thickness = wall.get('thickness', 0.2)
            
            # Calculate wall center and dimensions
            if start[0] != end[0]:  # Horizontal wall (along X)
                center_x = (start[0] + end[0]) / 2
                center_y = start[2] if len(start) > 2 else start[1]  # Handle both 2D and 3D coordinates
                center_z = room_height / 2
                scale_x = abs(end[0] - start[0]) / 2
                scale_y = thickness / 2
                scale_z = room_height / 2
            else:  # Vertical wall (along Y/Z)
                center_x = start[0]
                center_y = (start[2] + end[2]) / 2 if len(start) > 2 else (start[1] + end[1]) / 2
                center_z = room_height / 2
                scale_x = thickness / 2
                scale_y = abs(end[2] - start[2]) / 2 if len(end) > 2 else abs(end[1] - start[1]) / 2
                scale_z = room_height / 2
            
            lines += [
                f"bpy.ops.mesh.primitive_cube_add(location=({center_x}, {center_y}, {center_z}))",
                f"wall = bpy.context.active_object",
                f"wall.name = '{wall_id}'",
                f"wall.scale = ({scale_x}, {scale_y}, {scale_z})",
                "",
            ]

    # Add furniture
    lines.append("# Add furniture")
    for item in scene.get('furniture', []):
        item_id = item['id']
        
        # Get position from placements
        if item_id in placements:
            x, y = placements[item_id]
        else:
            # Fallback to original position
            print(f"Warning: {item_id} not found in placements, using original position")
            pos = item.get('xform', {}).get('pos', [1, 0, 1])
            x, y = pos[0], pos[2] if scene['metadata']['axes']['forward'] == 'Z' else pos[1]
        
        # Get dimensions
        dims = item.get('dims', {})
        width = dims.get('w', 1.0)
        depth = dims.get('d', dims.get('l', 1.0))  # 'd' or 'l' for depth/length
        height = dims.get('h', 0.5)
        
        # Calculate Z position (height from floor)
        z = height / 2  # Place object so bottom touches floor
        
        # Get rotation
        rot = item.get('xform', {}).get('rot', [0, 0, 0])
        
        # Create appropriate primitive based on category
        category = item.get('category', '').lower()
        
        if 'lamp' in category:
            # Create lamp as cylinder
            lines += [
                f"# {item_id} - {category}",
                f"bpy.ops.mesh.primitive_cylinder_add(location=({x:.2f}, {y:.2f}, {z:.2f}))",
                f"obj = bpy.context.active_object",
                f"obj.name = '{item_id}'",
                f"obj.scale = ({width/2:.2f}, {depth/2:.2f}, {height/2:.2f})",
                f"obj.rotation_euler = ({rot[0]}, {rot[1]}, {rot[2]})",
                "",
            ]
        else:
            # Create furniture as cube
            lines += [
                f"# {item_id} - {category}",
                f"bpy.ops.mesh.primitive_cube_add(location=({x:.2f}, {y:.2f}, {z:.2f}))",
                f"obj = bpy.context.active_object",
                f"obj.name = '{item_id}'",
                f"obj.scale = ({width/2:.2f}, {depth/2:.2f}, {height/2:.2f})",
                f"obj.rotation_euler = ({rot[0]}, {rot[1]}, {rot[2]})",
                "",
            ]

    # Add materials
    if 'materials' in scene:
        lines.append("# Create materials")
        for mat in scene['materials']:
            mat_id = mat['id']
            color = mat.get('color', [200, 200, 200])
            roughness = mat.get('roughness', 0.5)
            
            lines += [
                f"# Material: {mat_id}",
                f"mat = bpy.data.materials.new(name='{mat_id}')",
                f"mat.use_nodes = True",
                f"bsdf = mat.node_tree.nodes['Principled BSDF']",
                f"# Use input names instead of indices for better compatibility",
                f"bsdf.inputs['Base Color'].default_value = ({color[0]/255:.3f}, {color[1]/255:.3f}, {color[2]/255:.3f}, 1.0)",
                f"bsdf.inputs['Roughness'].default_value = {roughness}",
                "",
            ]
        
        # Apply materials to objects
        lines.append("# Apply materials")
        
        # Apply floor material
        if 'floorMat' in scene.get('room', {}):
            lines += [
                f"floor = bpy.data.objects.get('Floor')",
                f"if floor and '{scene['room']['floorMat']}' in bpy.data.materials:",
                f"    floor.data.materials.append(bpy.data.materials['{scene['room']['floorMat']}'])",
                "",
            ]
        
        # Apply wall materials
        for wall in scene.get('room', {}).get('walls', []):
            if 'mat' in wall:
                lines += [
                    f"wall = bpy.data.objects.get('{wall['id']}')",
                    f"if wall and '{wall['mat']}' in bpy.data.materials:",
                    f"    wall.data.materials.append(bpy.data.materials['{wall['mat']}'])",
                    "",
                ]

    # Add save command at the end
    lines += [
        "",
        "# Save the blend file",
        "import os",
        "os.makedirs('assets/renders', exist_ok=True)",
        "bpy.ops.wm.save_as_mainfile(filepath='assets/renders/scene.blend')",
        "",
        "# Render the scene",
        "bpy.context.scene.render.filepath = 'assets/renders/scene.png'",
        "bpy.ops.render.render(write_still=True)",
    ]

    print(f'Generated {len(lines)} lines of Blender Python code')

    with open(outfile, 'w') as f:
        f.write("\n".join(lines))
    
    return outfile

## 5  Headless Blender render

In [129]:
import subprocess, pathlib

def render_blend(pyfile: str, outfile: str='assets/renders/scene.blend'):
    blender_bin = '/Applications/Blender.app/Contents/MacOS/Blender'  # adjust path if needed
    # Just run the Python script which will save the blend file
    cmd = [blender_bin, '-b', '--python', pyfile]
    print(' '.join(cmd))
    subprocess.run(cmd, check=True)
    return pathlib.Path(outfile)

## 6  Vision‑language verifier (stub)

In [130]:
def verify_scene(image_path: str, scene: dict) -> float:
    """Return fraction of relations judged correct by GPT‑4V (stub)."""
    # TODO: implement with openai.Vision API once available.
    return 1.0  # assume perfect


In [131]:
def save_blender_code(code, filename="blender_scene.py"):
    """Save the generated Blender Python code to a file"""
    # Create directory if it doesn't exist
    os.makedirs("blender_scripts", exist_ok=True)
    
    # Full path to save the file
    filepath = os.path.join("blender_scripts", filename)
    
    # Write the code to file
    with open(filepath, "w") as f:
        f.write(code)
    
    print(f"Blender Python code saved to: {filepath}")
    return filepath

## 7  End‑to‑end pipeline

In [132]:
def plan_and_render(prompt: str):
    """
    Plan and render with automatic retry on constraint failures.
    After 2 attempts, continues with original positions if constraints fail.
    
    Args:
        prompt: Initial prompt for scene generation
        llm_to_scene: Function to convert prompt to scene JSON
        gen_blender_py: Function to generate Blender Python script
        render_blend: Function to render the Blender file
        verify_scene: Optional function to verify the scene
    """
    
    attempt = 0
    current_prompt = prompt
    last_error_msg = ""
    last_scene = None
    CONSTRAINT_RETRY_LIMIT = 2  # After this many tries, use original positions
    
    while attempt < MAX_RETRIES:
        attempt += 1
        print(f"\n{'='*50}")
        print(f"Attempt {attempt}/{MAX_RETRIES}")
        print(f"{'='*50}")
        
        try:
            # Generate scene from prompt
            print("Generating scene from prompt...")
            scene = llm_to_scene(current_prompt)
            last_scene = scene  # Save the last generated scene
            print('Scene generated:', scene)
            
            # Try to solve constraints
            print("\nSolving constraints...")
            placements = solve_scene(scene)
            
            if placements is False:
                # Constraint solving failed
                print(f"\n❌ Constraint solving failed on attempt {attempt}")
                
                if attempt < CONSTRAINT_RETRY_LIMIT:
                    # Try again with modified prompt
                    last_error_msg = "Previous attempt failed due to unsatisfiable constraints. "
                    current_prompt = f"{prompt}\n\nIMPORTANT: {last_error_msg}Please ensure furniture can fit in the room with proper spacing and avoid conflicting constraints."
                    
                    print(f"Waiting before retry...")
                    time.sleep(1)  # Brief pause before retry
                    continue
                else:
                    # After 2 tries, just use original positions from the scene
                    print(f"\n⚠️ Constraint solving failed after {CONSTRAINT_RETRY_LIMIT} attempts.")
                    print("Continuing with original positions from scene JSON...")
                    
                    # Extract original positions from the scene
                    placements = {}
                    for item in scene.get('furniture', []):
                        if 'id' in item and 'xform' in item and 'pos' in item['xform']:
                            # Use X,Y coordinates from original position
                            placements[item['id']] = (item['xform']['pos'][0], item['xform']['pos'][1])
                        else:
                            # Default position if not specified
                            placements[item['id']] = (1.0, 1.0)
                    
                    print(f"Using original placements: {placements}")
            
            print(f"\n✓ Proceeding with placements: {placements}")
            
            print("\nGenerating Blender script...")
            try:
                py_script = gen_blender_py(scene, placements) 
            except Exception as e:
                print(f"Error generating Blender script: {e}")
                if attempt < MAX_RETRIES:
                    current_prompt = f"{prompt}\n\nNote: Blender script generation failed. Please ensure valid scene structure."
                    continue
                else:
                    return None
            
            # Render with error handling
            print("Rendering scene...")
            try:
                blend_file = render_blend(py_script)
                
                if not blend_file:
                    raise ValueError("Render function returned empty result")
                    
            except Exception as e:
                print(f"Error during rendering: {e}")
                if attempt < MAX_RETRIES:
                    current_prompt = f"{prompt}\n\nNote: Rendering failed with error: {str(e)}. Please try a simpler scene."
                    continue
                else:
                    return None
            
            # Verify if function provided
            if verify_scene:
                try:
                    score = verify_scene(blend_file, scene)
                    print(f'Verification score: {score}')
                except Exception as e:
                    print(f"Warning: Verification failed with error: {e}")
                    # Don't fail the whole process if verification fails
            
            print(f"\nSuccess! Scene rendered successfully.")
            return blend_file
            
        except Exception as e:
            print(f"\nUnexpected error: {str(e)}")
            
            if attempt < MAX_RETRIES:
                # For non-constraint errors, also retry but with different message
                current_prompt = f"{prompt}\n\nNote: Previous attempt failed with error: {str(e)}. Please try a different approach."
                time.sleep(1)
                continue
            else:
                print(f"Failed after {MAX_RETRIES} attempts.")
                return None
    
    return None

## 8  Example run

In [133]:
example_prompt = "Create a bedroom with dimensions 4 x 2 x 3 meters. Include a bed, desk, and lamp. Position furniture logically with proper spacing and accessibility."

plan_and_render(example_prompt)


Attempt 1/5
Generating scene from prompt...
response ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "metadata": {\n    "projectId": "Bedroom1",\n    "roomType": "bedroom",\n    "units": "meters",\n    "axes": {\n      "right": "X",\n      "forward": "Y",\n      "up": "Z"\n    }\n  },\n  "room": {\n    "dims": {\n      "w": 4,\n      "d": 2,\n      "h": 3\n    },\n    "walls": [\n      {\n        "id": "wall1",\n        "start": [0, 0, 0],\n        "end": [4, 0, 0],\n        "thickness": 0.2,\n        "mat": "white",\n        "mesh": {}\n      },\n      {\n        "id": "wall2",\n        "start": [4, 0, 0],\n        "end": [4, 2, 0],\n        "thickness": 0.2,\n        "mat": "white",\n        "mesh": {}\n      },\n      {\n        "id": "wall3",\n        "start": [4, 2, 0],\n        "end": [0, 2, 0],\n        "thickness": 0.2,\n        "mat": "white",\n        "mesh": {}\n      },\n      {\n   

Traceback (most recent call last):
  File "/Users/divinirakiza/Workspaces/CALTECH/cs159/snapTo3D/3D_craft/scene.py", line 87, in <module>
    bsdf.inputs['Roughness'].default_value = None
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: bpy_struct: item.attr = val: NodeSocketFloatFactor.default_value expected a float type, not NoneType


PosixPath('assets/renders/scene.blend')